# Project 5 Investment cycles
### Authors Piotr Snopek Jakub Caputa

Y - national income
R - interest

In [1]:
from scipy.integrate import odeint

In [2]:
%matplotlib inline

import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
plt.style.use('seaborn-white')
plt.rcParams['figure.dpi'] = 100

In [3]:
from IPython.display import HTML
from ipywidgets.widgets import interact, IntSlider, FloatSlider, Layout

%matplotlib inline

style = {'description_width': '150px'}
slider_layout = Layout(width='99%')

# Keyness model

In [ ]:
def main(l0, l1, l2, l_s, alfa, beta1, beta2, beta3, beta, K, a1): 
    def I(y, r):
          return (l0 * K) / (l0 + (K - l0) * math.exp(-a1 * y))

    def S(y, r):
      return l1 * y + beta2 * r

    def L(y, r):
      return l2 * y - beta3 * r

    def f1(y, r):
      return I(y, r) - l1 * y - r * y * (beta1 + beta2)

    def f2(y, r):
      return r * (l2 * y - r * beta3 - l_s)

    def keynesian_model(params, t):
      y = params[0]
      r = params[1]
      dydt = f1(y, r) * alfa
      drdt = f2(y, r) * beta

      return [dydt, drdt]
    
    
    # initial condition
    params = [1.0, 0.1]

    # number of time points
    end = 1000

    # time points
    t = np.linspace(0, 100, end)

    # store solution
    y = np.empty_like(t)
    r = np.empty_like(t)
    Idt = np.empty_like(t)
    Sdt = np.empty_like(t)
    Ldt = np.empty_like(t)
    Mdt = np.empty_like(t)

    # record initial conditions
    y[0] = params[0]
    r[0] = params[1]
    Idt[0] = I(y[0], r[0])
    Sdt[0] = S(y[0], r[0])
    Ldt[0] = L(y[0], r[0])
    Mdt[0] = L(y[0], r[0]) - f2(y[0], r[0])

    # solve ODE
    for i in range(1, end):
        # span for next time step
        tspan = [t[i-1],t[i]]
        # solve for next step
        z = odeint(keynesian_model, params, tspan)
        # store solution for plotting
        y[i] = z[1][0]
        r[i] = z[1][1]
        Idt[i] = I(y[i], r[i])
        Sdt[i] = S(y[i], r[i])
        Ldt[i] = L(y[i], r[i])
        Mdt[i] = Ldt[i] - f2(y[i], r[i])
        # next initial condition
        params = z[1]
    
    T = 1000
    time = np.linspace(0, T, T)

    fig, ax = plt.subplots(figsize=(15, 10))

# uncomment this lines if you want to add additional information about the model    
    ax.plot(t, Idt, 'b-', label='I(t)')
    ax.plot(t, Sdt, 'r-', label='S(t)')
    ax.plot(t, Ldt, 'g-', label='L(t)')
    ax.plot(t, Mdt, 'y--', label='M(t)')
    ax.legend(loc='best', prop={'size': 15})
    plt.show()

interact(main, alfa=FloatSlider(min=0, max=10, step=0.01, value=2, description='alfa', style=style, layout=slider_layout),
               beta=FloatSlider(min=0, max=10, step=0.01, value=7, description='beta', style=style, layout=slider_layout),
               l0=FloatSlider(min=0, max=0.1, step=0.001, value=0.005, description='l0', style=style, layout=slider_layout),
               l1=FloatSlider(min=0, max=3, step=0.01, value=0.0, description='l1', style=style, layout=slider_layout),
               l2=FloatSlider(min=0, max=3, step=0.01, value=0.35, description='l2', style=style, layout=slider_layout),
               l_s=FloatSlider(min=0, max=1.5, step=0.01, value=0.2, description='l_s', style=style, layout=slider_layout),
               beta1=FloatSlider(min=0, max=2, step=0.01, value=1.6, description='beta1', style=style, layout=slider_layout),
               beta2=FloatSlider(min=0, max=1.5, step=0.01, value=0.4, description='beta2', style=style, layout=slider_layout),
               beta3=FloatSlider(min=0, max=1.5, step=0.01, value=1, description='beta3', style=style, layout=slider_layout),
               K=FloatSlider(min=0, max=10, step=0.01, value=1, description='K', style=style, layout=slider_layout),
               a1=FloatSlider(min=0, max=6, step=0.01, value=4, description='a1', style=style, layout=slider_layout)
         
        );

# Model 9d

In [105]:
T = 150
t = np.linspace(0, T, T)

F = np.empty_like(t)
H = np.empty_like(t)
K = np.empty_like(t)
L = np.empty_like(t)
M = np.empty_like(t)
p = np.empty_like(t)
w = np.empty_like(t)
G_INV = np.empty_like(t)
r = np.empty_like(t)

C = np.empty_like(t)
S = np.empty_like(t)
I = np.empty_like(t)
Div = np.empty_like(t)


def main(Q1, Q2, Q3, ALPHA_R,ALPHA_P,ALPHA_M,ALPHA_F,ALPHA_INV,T_WAGE,T_EMPL,A,LAMBDA,Y_SAVE,T_DEP,G_MAX,L_MAX):
    def Model(params,t):
        F, M, H, K, L, p, w, G_INV, r = params
        if L_MAX < L:
            L_MAX_NEW = L
        else:
            L_MAX_NEW = L_MAX
    
        C = (1 - Y_SAVE * (1 + r)) * (1/p) * ALPHA_M * M # ok
        I = G_INV * (1/p) * ALPHA_F * F # ok
        D = C + I # ok
        Y = A * math.pow(L, LAMBDA) * math.pow(K, 1 - LAMBDA)
        Ld = (8 * math.pow(p, 3) * K * math.pow(A, 3))/(27 * math.pow(w, 3)) # ok
        Le = min(L_MAX_NEW, Ld)
        S = Y_SAVE * (1 + r) * ALPHA_M * M
        PI = p * D - w * L
        Div = ALPHA_F * F - p * I
        PIn = PI - (1 / T_DEP) * p * K

        # to jest ok
        dFdt = PI + S - Div - (p * I)
        dHdt = Y - D
        dKdt = (-1/T_DEP) * K  + I
        dLdt = (-1/T_EMPL) * (L - Le)
        dMdt = ((w * L) + Div) - ((p * C) + S)
        dpdt =  -1 * ((p * ALPHA_P * H)/D)

        dwdt = (w/T_WAGE) * (Ld - L)

        if PIn/(p * K) >= r:
            dG_INVdt = ALPHA_INV * (G_MAX - G_INV) * ( ( PIn/(p * K) ) - r)
        else:
            dG_INVdt = ALPHA_INV * G_INV * ( ( PIn/(p * K) ) - r)

        drdt = ALPHA_R * r * (((w * L) + Div) - ((p * C ) + S))

        return dFdt, dMdt, dHdt, dKdt, dLdt , dpdt, dwdt, dG_INVdt, drdt
    
    F[0], M[0], H[0], K[0], L[0], p[0], w[0], G_INV[0], r[0] = 37, 58, 0, 43.5, 163.6, 1.56, 0.19, 0.46, 0.025
    params = [F[0],M[0],H[0],K[0],L[0],p[0], w[0], G_INV[0],r[0]]

    
    C[0] = (1 - Y_SAVE * (1 + r[0])) * (1/p[0]) * ALPHA_M * M[0]
    S[0] = Y_SAVE * (1 + r[0]) * ALPHA_M * M[0]
    I[0] = G_INV[0] * (1/p[0]) * ALPHA_F * F[0]
    Div[0] = ALPHA_F * F[0] - p[0] * I[0]
    
    for i in range(1, T):
        tspan = [t[i-1],t[i]]
        result = odeint(Model, params, tspan)
        F[i], M[i], H[i], K[i], L[i] , p[i], w[i], G_INV[i], r[i] = result[1]
        
        C[i] = ((1 - Y_SAVE * (1 + r[i])) * (1/p[i]) * ALPHA_M * M[i])
        S[i] = (Y_SAVE * (1 + r[i]) * ALPHA_M * M[i])
        I[i] = (G_INV[i] * (1/p[i]) * ALPHA_F * F[i])
        Div[i] = (ALPHA_F * F[i] - p[i] * I[i])

        params = result[1]
        
        
def plot_model(Q1, Q2, Q3, ALPHA_R,ALPHA_P,ALPHA_M,ALPHA_F,ALPHA_INV,T_WAGE,T_EMPL,A,LAMBDA,Y_SAVE,T_DEP,G_MAX,L_MAX): 
    
    
    # Starting Params
    main(Q1, Q2, Q3, ALPHA_R,ALPHA_P,ALPHA_M,ALPHA_F,ALPHA_INV,T_WAGE,T_EMPL,A,LAMBDA,Y_SAVE,T_DEP,G_MAX,L_MAX)
    
    fig, ax = plt.subplots(figsize=(15, 9))

    ax.plot(t, F, label='F(t) Liquid assets of banks and firms ')
    ax.plot(t, M, label='M(t) Money holdings of consumers ')
    ax.plot(t, K, label='K(t) Capital')
    ax.plot(t, L, label='L(t) Number of employed')
    ax.plot(t, H, label='H(t) Commodity inventories') # zapasy towarów
    ax.plot(t, w*100, label='w(t) Mean wage') 
    ax.legend(loc='best', prop={'size': 15})
    ax.grid()
    plt.show()
    
    print('Investments are inversely related to interest.  They increase when interest is low, national income and savings  accordingly  grow,  and  interest  starts  decreasing  to  stimulate  the  transition  from    S    to   I.    Higher  interest  puts a brake on investment activity, which in turn leads to reduction of national income.  If the interest is set too high, money holders will purchase securities.  This leads to an increase in the demand for bonds and their prices rise.  When money supply exceeds the demand for money, the interest decreases.')

    fig, ax = plt.subplots(figsize=(15, 9))
    ax.plot(t, G_INV, label='Ginv(t) Investment level')
    ax.plot(t, r, label='r(t) Interest')
    ax.plot(t, p, label='p(t) Price level')
    ax.legend(loc='best', prop={'size': 15})
    ax.grid()
    plt.show()
    
    fig, ax = plt.subplots(figsize=(15, 9))
    ax.plot(t, C, label='C(t) Consumption')
    ax.plot(t, S, label='S(t) Savings')
    ax.plot(t, I, label='I(t) Investments')
    ax.plot(t, Div, label='Div(t) Dividends')
    ax.legend(loc='best', prop={'size': 15})
    ax.grid()
    plt.show()
    

interact(plot_model, Q1=FloatSlider(min=0, max=100, step=0.5, value=95, description='Q1', style=style, layout=slider_layout),
               Q2=FloatSlider(min=0, max=100, step=0.5, value=95, description='Q2', style=style, layout=slider_layout),
               Q3=FloatSlider(min=0, max=100, step=0.5, value=82.635, description='Q3', style=style, layout=slider_layout),
               ALPHA_R=FloatSlider(min=0, max=3, step=0.01, value=0.1, description='ALPHA_R', style=style, layout=slider_layout),
               ALPHA_P=FloatSlider(min=0, max=1.5, step=0.01, value=0.036, description='ALPHA_P', style=style, layout=slider_layout),
               ALPHA_M=FloatSlider(min=0, max=1.5, step=0.01, value=0.8, description='ALPHA_M', style=style, layout=slider_layout),
               ALPHA_F=FloatSlider(min=0, max=1.5, step=0.01, value=0.8, description='ALPHA_F', style=style, layout=slider_layout),
               ALPHA_INV=FloatSlider(min=0, max=3, step=0.01, value=2, description='ALPHA_INV', style=style, layout=slider_layout),  
               T_WAGE=FloatSlider(min=0, max=1.5, step=0.01, value=0.1, description='T_WAGE', style=style, layout=slider_layout), 
               T_EMPL=FloatSlider(min=0, max=1.5, step=0.01, value=0.1, description='T_EMPL', style=style, layout=slider_layout),  
               A=FloatSlider(min=0, max=1.5, step=0.01, value=9.3 * 0.03, description='A', style=style, layout=slider_layout),  
               LAMBDA=FloatSlider(min=0, max=1, step=0.01, value=1, description='LAMBDA', style=style, layout=slider_layout),  
               Y_SAVE=FloatSlider(min=0, max=1.5, step=0.01, value=0.3, description='Y_SAVE', style=style, layout=slider_layout),  
               T_DEP=FloatSlider(min=0, max=6, step=0.01, value=5, description='T_DEP', style=style, layout=slider_layout),  
               G_MAX=FloatSlider(min=0, max=1.5, step=0.01, value=0.8, description='G_MAX', style=style, layout=slider_layout),  
               L_MAX=FloatSlider(min=0, max=300, step=1, value=200, description='L_MAX', style=style, layout=slider_layout)
        );

interactive(children=(FloatSlider(value=95.0, description='Q1', layout=Layout(width='99%'), step=0.5, style=Sl…

In [5]:
!pip install SALib

In [6]:
from SALib.sample import saltelli
from SALib.analyze import sobol
from SALib.test_functions import Ishigami
import numpy as np

In [82]:
params_start = [95, 95, 82.635, 0.1, 0.036, 0.8, 0.8, 2, 0.1, 0.1, 9.3*0.03, 1, 0.3, 5, 0.8, 200]
params_90 = [x * 0.9 for x in params_start]
params_110 = [x * 1.1 for x in params_start]
problem = {
    'num_vars': 16,
    'names': ['Q1', 'Q2', 'Q3', 'ALPHA_R','ALPHA_P','ALPHA_M','ALPHA_F','ALPHA_INV','T_WAGE','T_EMPL','A','LAMBDA','Y_SAVE','T_DEP','G_MAX','L_MAX'],
    'bounds': list(map(list, zip(params_90,params_110)))
}


In [83]:
param_values = saltelli.sample(problem=problem, N=8, calc_second_order=False)

In [84]:
import pandas as pd

df = pd.DataFrame(param_values)
pd.set_option('display.max_rows', df.shape[0]+1)
df.head(1000)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,87.28125,94.40625,82.118531,0.103125,0.034425,0.875,0.805,2.1375,0.099375,0.093125,0.256331,0.98125,0.309375,5.15625,0.775,181.25
1,100.34375,94.40625,82.118531,0.103125,0.034425,0.875,0.805,2.1375,0.099375,0.093125,0.256331,0.98125,0.309375,5.15625,0.775,181.25
2,87.28125,96.78125,82.118531,0.103125,0.034425,0.875,0.805,2.1375,0.099375,0.093125,0.256331,0.98125,0.309375,5.15625,0.775,181.25
3,87.28125,94.40625,87.283219,0.103125,0.034425,0.875,0.805,2.1375,0.099375,0.093125,0.256331,0.98125,0.309375,5.15625,0.775,181.25
4,87.28125,94.40625,82.118531,0.090625,0.034425,0.875,0.805,2.1375,0.099375,0.093125,0.256331,0.98125,0.309375,5.15625,0.775,181.25
5,87.28125,94.40625,82.118531,0.103125,0.037575,0.875,0.805,2.1375,0.099375,0.093125,0.256331,0.98125,0.309375,5.15625,0.775,181.25
6,87.28125,94.40625,82.118531,0.103125,0.034425,0.865,0.805,2.1375,0.099375,0.093125,0.256331,0.98125,0.309375,5.15625,0.775,181.25
7,87.28125,94.40625,82.118531,0.103125,0.034425,0.875,0.815,2.1375,0.099375,0.093125,0.256331,0.98125,0.309375,5.15625,0.775,181.25
8,87.28125,94.40625,82.118531,0.103125,0.034425,0.875,0.805,2.0375,0.099375,0.093125,0.256331,0.98125,0.309375,5.15625,0.775,181.25
9,87.28125,94.40625,82.118531,0.103125,0.034425,0.875,0.805,2.1375,0.100625,0.093125,0.256331,0.98125,0.309375,5.15625,0.775,181.25


In [91]:
from tqdm import tqdm
T=100
Y_C = np.zeros([param_values.shape[0]])
Y_S = np.zeros([param_values.shape[0]])
Y_I = np.zeros([param_values.shape[0]])
Y_DIV = np.zeros([param_values.shape[0]])
for i, X in tqdm(enumerate(param_values)):
    main(*X)
    Y_C[i] = C[T-1]
    Y_S[i] = S[T-1]
    Y_I[i] = I[T-1]
    Y_DIV[i] = Div[T-1]

144it [00:50,  2.87it/s]


In [86]:
Y



array([2.54021951e+01, 2.54021951e+01, 2.54021951e+01, 2.54021951e+01,
       2.53644343e+01, 2.32998691e+01, 2.54140088e+01, 2.53593686e+01,
       2.41055282e+01, 2.54027979e+01, 2.54021951e+01, 2.57772748e+01,
       2.01855281e+01, 2.50061627e+01, 2.23605582e+01, 2.50078365e+01,
       2.54021951e+01, 1.99829198e+01, 2.20133180e+01, 2.20133180e+01,
       2.20133180e+01, 2.20133180e+01, 2.16049914e+01, 2.46181546e+01,
       2.18725606e+01, 2.23754840e+01, 1.99885229e+01, 2.19177955e+01,
       2.20133180e+01, 2.21795320e+01, 2.48254344e+01, 2.28375099e+01,
       2.02390193e+01, 2.31566727e+01, 2.20133180e+01, 2.63605370e+01,
       1.53207017e+01, 1.53207017e+01, 1.53207017e+01, 1.53207017e+01,
       1.78323721e+01, 2.50419060e-04, 1.44717152e+01, 1.46083701e+01,
       2.32214792e+01, 1.53213772e+01, 1.53207017e+01, 1.29329394e+01,
       2.69810809e+01, 1.89785743e+01, 2.12044985e+01, 2.58006101e+01,
       1.53205665e+01, 2.56879611e+01, 2.61992716e+01, 2.61992716e+01,
      

In [87]:
np.argwhere(np.isnan(Y))

array([], shape=(0, 1), dtype=int64)

In [92]:
Si_C = sobol.analyze(problem, Y_C, calc_second_order=False)
Si_S = sobol.analyze(problem, Y_S, calc_second_order=False)
Si_I = sobol.analyze(problem, Y_I, calc_second_order=False)
Si_DIV = sobol.analyze(problem, Y_DIV, calc_second_order=False)

In [89]:
Si

{'S1': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  2.24124353e-02,
         7.61134836e-02, -1.10998900e-02, -1.19681532e-02,  2.11864982e-01,
        -2.34247023e-03,  1.20978053e-04,  1.61209266e-01, -1.72789259e-01,
         3.87304762e-02,  2.46593750e-01,  1.85657735e-01, -1.49597685e-06]),
 'S1_conf': array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 5.01822629e-02,
        5.97872673e-01, 1.58220482e-02, 2.29573757e-02, 2.16278240e-01,
        5.43626844e-03, 9.50460942e-04, 2.78007652e-01, 1.44169144e+00,
        1.01600061e-01, 2.52920443e-01, 4.09090161e-01, 2.41040793e-06]),
 'ST': array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.27235660e-02,
        8.86685113e-01, 2.61364637e-03, 2.32563763e-03, 2.52593783e-01,
        3.34013884e-05, 2.01904629e-07, 1.56117261e-01, 1.01777165e+00,
        5.37607830e-02, 1.78785710e-01, 4.68796781e-01, 7.49111827e-11]),
 'ST_conf': array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.60016945e-02,
        

In [97]:
np.set_printoptions(suppress=True)
Si_C['S1']

array([ 0.        ,  0.        ,  0.        ,  0.02241244,  0.07611348,
       -0.01109989, -0.01196815,  0.21186498, -0.00234247,  0.00012098,
        0.16120927, -0.17278926,  0.03873048,  0.24659375,  0.18565773,
       -0.0000015 ])

In [98]:
Si_S['S1']

array([ 0.        ,  0.        ,  0.        ,  0.09410423, -1.06264308,
       -0.02776483, -0.03401737,  0.28966363,  0.00043593,  0.00005264,
       -0.03729839,  0.34869984,  0.06347308,  0.24028507,  0.48594999,
       -0.00000084])

In [99]:
Si_I['S1']

array([ 0.        ,  0.        ,  0.        , -0.02045258, -0.11851903,
       -0.00833117, -0.02341215,  0.09098444, -0.00029665,  0.00011477,
        0.03833102,  0.311345  ,  0.07037827,  0.11720994,  0.55150992,
        0.00000038])

In [100]:
Si_DIV['S1']

array([ 0.        ,  0.        ,  0.        , -0.06957908, -0.04393387,
       -0.0056334 , -0.02695118, -0.24260888, -0.00307412, -0.00010205,
        0.01845371, -0.06932199, -0.02058629, -0.24074875,  0.04277688,
        0.00000014])

In [103]:
params_avg = (Si_C['S1'] + Si_S['S1'] + Si_I['S1'] + Si_DIV['S1']) /4
params_avg

array([ 0.        ,  0.        ,  0.        ,  0.00662125, -0.28724563,
       -0.01320732, -0.02408721,  0.08747604, -0.00131933,  0.00004659,
        0.0451739 ,  0.1044834 ,  0.03799888,  0.090835  ,  0.31647363,
       -0.00000045])